In [ ]:
%pip install -q -U ollama

In [ ]:
import os
import json

from ollama import Client as Ollama

In [ ]:
OLLAMA_ENDPOINT = "http://192.168.4.5:11434"
OLLAMA_MODEL = "llava:34b" # llava:7b, llava:13b, llava:34b
INPUT_DIR = "./input"
OUTPUT_DIR = "./output"

PROMPT = "You are a helpful assistant tasked with suggesting a relevant filename based on the content of a given image. Your response should be a raw JSON object with a single key-value pair. The key should be 'filename' and the value should be the suggested name for the file without an extension. Do not include any additional information or explanations in your response, just return the filename in the required format."

In [ ]:
def get_image_files(dir):
    """
    Returns a list of all image files (jpg, jpeg, png) in the given directory.

    Args:
        directory (str): The path to the directory to search for image files.

    Returns:
        list: A list of strings, each representing an absolute file path to an image file.
    """
    image_types = ['jpg', 'jpeg', 'png']
    image_files = []

    # Loop through all files in the given directory
    for filename in os.listdir(dir):
        # Check if the file has an image type extension
        file_ext = filename.split('.')[-1]
        if file_ext.lower() in image_types:
            # If so, add its absolute path to the list of image files
            image_files.append(os.path.join(dir, filename))

    return image_files

In [ ]:
def generate_filename(file_path):
    ollama = Ollama(host=OLLAMA_ENDPOINT)
    try:
        resp = ollama.chat(
            model=OLLAMA_MODEL,
            messages=[
                {
                    'role': 'user',
                    'content': PROMPT,
                    'images': ["./image.png"]
                },
                {
                    'role': 'assistant',
                    'content': '{ "filename": "a-blue-sports-car-driving-in-the-mountains" }'
                },
                {
                    'role': 'user',
                    'content': PROMPT,
                    'images': [ file_path ]
                }
            ]
        )
        response = json.loads(resp['message']['content'])
        filename = response['filename']
        return filename
    except Exception as err:
        print(f"Error processing {file_path}: {err}")
        print()

In [ ]:
image_files = get_image_files(INPUT_DIR)

for file in image_files:
    file_ext = file.split('.')[-1]
    filename = generate_filename(file)
    new_filename = f"{OUTPUT_DIR}/{filename}.{file_ext}"
    print(f"Old Filename: {file}")
    print(f"New Filename: {new_filename}")
    print()